# Centroid based summerisation

In [1]:
import os
import codecs
import warnings
warnings.filterwarnings("ignore")

corpus_path = "news-corpus//"
article_paths = [os.path.join(corpus_path,p) for p in os.listdir(corpus_path)]

doc_complete = []
for path in article_paths:
    with open(path, 'rb') as f:
        doc_content = f.read().decode(errors='ignore')
        doc_complete.append(doc_content)

In [2]:
import re
for i in range(len(doc_complete)):
    doc_complete[i] = re.sub(r'[^\w\s.]', '', doc_complete[i])

In [3]:
if len(doc_complete) >= 2:
    doc_complete.pop(1)

In [4]:
len(doc_complete)

5

In [8]:
#doc_complete[1]

In [17]:
# import nltk
# from nltk.tokenize import sent_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cluster import KMeans
# from sklearn.metrics.pairwise import cosine_distances

# # nltk.download('punkt')
# # nltk.download('stopwords')
# # nltk.download('wordnet')


# # Preprocess the text data
# stopwords = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# # Tokenize and preprocess each document
# preprocessed_documents = []
# for document in doc_complete:
#     # Tokenize the sentences in the document
#     sentences = sent_tokenize(document)

#     # Preprocess each sentence
#     preprocessed_sentences = []
#     for sentence in sentences:
#         # Tokenize the sentence
#         tokens = nltk.word_tokenize(sentence.lower())

#         # Remove stopwords and punctuation
#         tokens = [token for token in tokens if token.isalpha() and token not in stopwords]

#         # Lemmatize the tokens
#         tokens = [lemmatizer.lemmatize(token) for token in tokens]

#         preprocessed_sentences.append(" ".join(tokens))

#     preprocessed_documents.append(preprocessed_sentences)

# # Flatten the preprocessed documents
# flattened_documents = [" ".join(sentences) for sentences in preprocessed_documents]

# # Vectorize the preprocessed documents (TF-IDF)
# vectorizer = TfidfVectorizer()
# vectorized_docs = vectorizer.fit_transform(flattened_documents)

# # Perform clustering (K-means)
# num_clusters = 5  # Adjust the number of clusters as needed
# kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=42)
# kmeans.fit(vectorized_docs)
# cluster_labels = kmeans.labels_

# # Calculate the centroid of each cluster
# centroids = kmeans.cluster_centers_

# # Calculate cosine distances between documents and centroids
# distances = cosine_distances(vectorized_docs, centroids)

# # Sort documents by distance to centroids
# sorted_indices = distances.argsort(axis=0).ravel()

# # Select top documents for summary
# summary_documents = [flattened_documents[i] for i in sorted_indices[:6]]  # Adjust the number of documents for summary

# # Print the summaries
# for i, document in enumerate(summary_documents):
#     print(f"Summary {i + 1}:")
#     print(document)
#     print("==========================")


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances

def centroid_summarization(text, num_sentences):
    
    # Tokenize the text into sentences
    sentences = text.split('. ')  # Assuming sentences are separated by '.'

    # Vectorize the sentences using TF-IDF
    vectorizer = TfidfVectorizer()
    sentence_vectors = vectorizer.fit_transform(sentences)

    # Calculate the centroid vector
    centroid = sentence_vectors.mean(axis=0)

    # Calculate the cosine distances between each sentence vector and the centroid
    distances = cosine_distances(sentence_vectors, centroid)

    # Sort sentences based on their distances to the centroid
    sorted_sentences = sorted(enumerate(sentences), key=lambda x: distances[x[0]], reverse=False)

    # Select the top 'num_sentences' sentences as the summary
    summary = [sentence for _, sentence in sorted_sentences[:num_sentences]]

    # Join the summary sentences into a single string
    summary_text = '. '.join(summary)

    return summary_text

generated_summaries = []
for doc in doc_complete:
    
    num_sentences = 3

    summary = centroid_summarization(doc, num_sentences)
    print(summary)
    generated_summaries.append(summary)
    print("\n\n")
    print("---------------------------------------------------------------------------------------------")
    print("\n\n")


At a time when the two extremes of malnourishment and obesity plague large portions of the world India has taken it upon herself to educate the masses about these smallseeded grasses that are highly beneficial to human health.Millet and grain cereals despite being rich sources of protein and antioxidants with high nutritional value have never been considered fashionable foods however India has done remarkably well when it has come to meeting the caloric needs and demands of her people.India the worlds largest producer and the worlds secondlargest exporter of millet are hoping to change the humble millets reputation worldwide.Unlike a large part of the rest of the world almost every Indian household is acquainted with the taste and the benefits of millet.Millets have been a staple of the Indian diet especially in rural India for years and remain prevalent even today. It was a remarkable achievement considering India had not even touched the export mark of USD 30 million worth of millets

In [6]:
def extract_text_between_markers(text, start_marker, end_marker):
    extracted_texts = []
    start_index = 0

    while True:
        start_index = text.find(start_marker, start_index)
        if start_index == -1:
            break

        start_index += len(start_marker)
        end_index = text.find(end_marker, start_index)
        if end_index == -1:
            break

        extracted_text = text[start_index:end_index].strip()
        extracted_texts.append(extracted_text)

        start_index = end_index + len(end_marker)

    return extracted_texts

In [7]:
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

reference_summaries =[]
file_path = "News articles.docx"
text_content = read_docx(file_path)


start_marker = "Summary:"
end_marker = "Top Sentences:"

extracted_texts = extract_text_between_markers(text_content, start_marker, end_marker)
for extracted_text in extracted_texts:
    reference_summaries.append(extracted_text)

In [8]:
reference_summaries

["The Indian government, led by Agriculture Minister Narendra Singh Tomar, has opened the 'Millets Experience Centre' in New Delhi to promote the production and consumption of millets. The center aims to raise awareness about the nutritional benefits of millets and encourage their adoption among the public. Tomar emphasized the health advantages of millets and their climate resilience, highlighting that increased millet production would benefit farmers and startups in the sector. The United Nations has declared 2023 as the International Year of Millets, further positioning India as a leader in promoting this crop.",
 "India is taking initiatives to educate people about the nutritional value of millets and promote their consumption. Despite being rich in protein and antioxidants, millets have not been considered fashionable foods. However, the Indian government has recognized millets as a reliable grain for enhancing farmers' income and ensuring food security. Efforts to revive millet p

# Bleu Scores

In [11]:
from nltk.translate.bleu_score import sentence_bleu

bleu_score = sentence_bleu(generated_summaries[1],reference_summaries[0])
print("BLEU score for Article",":", bleu_score)


BLEU score for Article : 8.548028765846341e-232


In [12]:
bleu_score = sentence_bleu(generated_summaries[2],reference_summaries[1])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 8.58290516843769e-232


In [16]:
bleu_score = sentence_bleu(generated_summaries[3],reference_summaries[2])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 9.318317688234048e-232


In [17]:
bleu_score = sentence_bleu(generated_summaries[1],reference_summaries[3])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 8.513542039802197e-232


In [18]:
bleu_score = sentence_bleu(generated_summaries[4],reference_summaries[4])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 9.404418754951175e-232
